# ERC-4626: vault ecosystem comparison across chains

This notebook serves both as a coding tutorial and a useful data analytics tool for ERC-4626 vaults. 

- In this notebook, we examine different ERC-4626 vaults across different EVM blockchains   
    - Currently we do not scan non-ERC-4626 vaults like Enzyme Finance, or any protocol-native vaults like Hyperliquid HPL. This is not an inherit limitation, this is not just yet implemented.
- We assemble various data tables out of the vault data to show and compare the blockchain ecosystems
- The analysis focus on USD-stablecoin nonminatd vaults
    - Currently missing are e.g. WETH vaults and staking vaults for various small cap tokens
    - There is no ERC standard for vaults fees - for some protocols we have manualled added fee reading support  
- The list of chains is somewhat randomly selected and very easy to extend to contain any chain supported by [Envio's HyperSync](https://docs.envio.dev/docs/HyperSync/hypersync-supported-networks)
- Everything is open source: You can run this notebook and associated scripts yourself on your local computer, it will take around an hour

In this notebook, we use terms Net Asset Value (NAV) and [Total Value Locked (TVL)](https://tradingstrategy.ai/glossary/total-value-locked-tvl) interchangeably.

## Usage

- Read general instructions [how to run the tutorials](./)
- See `ERC-4626 scanning all vaults onchain` example in tutorials first how to build a vault database as local `vault_db.pickle` file.




## Setup

- Set up notebook renderinb parmaeters

In [1]:
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format

## Read scanned data

- Read the Pickle database our scanning script produced earlier 

In [2]:
import pickle
from pathlib import Path

import pandas as pd

from eth_defi.token import is_stablecoin_like

output_folder = Path("~/.tradingstrategy/vaults").expanduser()
vault_db = output_folder / "vault-db.pickle"
assert vault_db.exists(), "Run the vault scanner script first"

vault_db = pickle.load(open(vault_db, "rb"))

print(f"We have data for {len(vault_db)} vaults")

We have data for 7054 vaults


## Transform data

- Prepare the raw vault pickled data as Pandas DataFrame for data research

In [3]:
import datetime
from pprint import pformat
import pandas as pd
from eth_defi.erc_4626.hypersync_discovery import ERC4262VaultDetection
from eth_defi.chain import get_chain_name
from eth_defi.token import is_stablecoin_like

data = list(vault_db.values())
df = pd.DataFrame(data)

# print("Raw row example:")
# print(df.iloc[0])

# Build useful columns out of raw pickled Python data
# _detection_data contains entries as ERC4262VaultDetection class
entry: ERC4262VaultDetection
df["Chain"] = df["_detection_data"].apply(lambda entry: get_chain_name(entry.chain))
df["Protocol identified"] = df["_detection_data"].apply(lambda entry: entry.is_protocol_identifiable())
df["Stablecoin denominated"] = df["_denomination_token"].apply(lambda token_data: is_stablecoin_like(token_data["symbol"]) if pd.notna(token_data) else False)
df["ERC-7540"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7540())
df["ERC-7575"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7575())
df["Fee detected"] = df.apply(lambda row: (row["Mgmt fee"] is not None) or (row["Perf fee"] is not None), axis=1)
# Event counts
df["Deposit count"] = df["_detection_data"].apply(lambda entry: entry.deposit_count)
df["Redeem count"] = df["_detection_data"].apply(lambda entry: entry.redeem_count)
df["Total events"] = df["Deposit count"] + df["Redeem count"]
df["Mgmt fee"] = df["Mgmt fee"].fillna("<unknown>")
df["Perf fee"] = df["Mgmt fee"].fillna("<unknown>")
df["Age"] = datetime.datetime.utcnow() - df["First seen"]
df["NAV"] = df["NAV"].astype("float64")

df = df.sort_values(["Chain", "Address"])
df = df.set_index(["Chain", "Address"])

print("DataFrame MultiIndex is:", ", ".join(x for x in df.index.names))
print("DataFrame columns are:", ", ".join(x for x in df.columns))

source_df = df

display(df.sort_values("Total events", ascending=False).head())



DataFrame MultiIndex is: Chain, Address
DataFrame columns are: Symbol, Name, Denomination, NAV, Protocol, Mgmt fee, Perf fee, Shares, First seen, _detection_data, _denomination_token, _share_token, Protocol identified, Stablecoin denominated, ERC-7540, ERC-7575, Fee detected, Deposit count, Redeem count, Total events, Age


Symbol  \
Chain     Address                                                   
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0     yvUSDC-A   
          0xbb287e6017d3deb0e2e65061e8684eab21060123     yvUSDT-A   
Ethereum  0xd9a442856c234a39a81a089c06451ebaa4306a72       pufETH   
Base      0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1        mwETH   
Berachain 0x90bc07408f5b5eac4de38af76ea6069e1fcee363  USDC.eVault   

                                                                              Name  \
Chain     Address                                                                    
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0                 USDC yVault-A   
          0xbb287e6017d3deb0e2e65061e8684eab21060123                 USDT yVault-A   
Ethereum  0xd9a442856c234a39a81a089c06451ebaa4306a72                        pufETH   
Base      0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1         Moonwell Flagship ETH   
Berachain 0x90bc07408f5b5eac4de38af76ea6069e1fcee363  Bridged USDC (Stargate)Vault   

                                                     Denomination  \
Chain     Address                                                   
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0         USDC   
          0xbb287e6017d3deb0e2e65061e8684eab21060123         USDT   
Ethereum  0xd9a442856c234a39a81a089c06451ebaa4306a72         WETH   
Base      0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1         WETH   
Berachain 0x90bc07408f5b5eac4de38af76ea6069e1fcee363       USDC.e   

                                                                NAV  \
Chain     Address                                                     
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0     570,858.69   
          0xbb287e6017d3deb0e2e65061e8684eab21060123     462,288.76   
Ethereum  0xd9a442856c234a39a81a089c06451ebaa4306a72      69,830.82   
Base      0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1      13,396.42   
Berachain 0x90bc07408f5b5eac4de38af76ea6069e1fcee363 307,578,099.32   

                                                                Protocol  \
Chain     Address                                                          
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0            Yearn v3   
          0xbb287e6017d3deb0e2e65061e8684eab21060123            Yearn v3   
Ethereum  0xd9a442856c234a39a81a089c06451ebaa4306a72  <unknown ERC-4626>   
Base      0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1              Morpho   
Berachain 0x90bc07408f5b5eac4de38af76ea6069e1fcee363  <unknown ERC-4626>   

                                                       Mgmt fee   Perf fee  \
Chain     Address                                                            
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0  <unknown>  <unknown>   
          0xbb287e6017d3deb0e2e65061e8684eab21060123  <unknown>  <unknown>   
Ethereum  0xd9a442856c234a39a81a089c06451ebaa4306a72  <unknown>  <unknown>   
Base      0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1       0.00       0.00   
Berachain 0x90bc07408f5b5eac4de38af76ea6069e1fcee363  <unknown>  <unknown>   

                                                                            Shares  \
Chain     Address                                                                    
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0                  484583.69304   
          0xbb287e6017d3deb0e2e65061e8684eab21060123                 390039.532892   
Ethereum  0xd9a442856c234a39a81a089c06451ebaa4306a72      66891.106108053732160081   
Base      0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1      13269.176227251264208125   
Berachain 0x90bc07408f5b5eac4de38af76ea6069e1fcee363  307578099.320120430232545112   

                                                              First seen  \
Chain     Address                                                          
Polygon   0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0 2023-10-26 16:45:48   
          0xbb287e6017d3deb0e2e65061e8684eab21060123 2023-12-07 17:02:20   
Eth

## Vaults per chain summary

- Get a summary of scanned chains at what vaults they have
- *Generic* status means that we do not have classification rules to determine the protocol on which a particular ERC-4626 vault belongs
- *Broken* status means that we could not correctly extract ERC-4626 information out of a smart contract

To detect the protocol of a vault, we need to maintain a [manual rule list here](https://github.com/tradingstrategy-ai/web3-ethereum-defi/blob/master/eth_defi/erc_4626/classification.py). Not all protocols are supported at the moment. as there are too many protocols to manually examine and identify them. Open source contributions welcome.




In [4]:
nav_threshold = 100_000
broken_max_nav_threshold = 999_000_000_000

# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True 
erc_7575 = df["ERC-7575"] == True 
fee_detected = df["Fee detected"] == True 

stablecoin_only_df = df[df["Stablecoin denominated"] == True]
stablecoin_only_df = stablecoin_only_df[stablecoin_only_df["NAV"] < broken_max_nav_threshold]

# Create the summary DataFrame
summary_df = pd.DataFrame({
    'Total vaults detected': df.groupby(level='Chain').size(),
    'Total vault TVL/NAV in stable vaults': stablecoin_only_df.groupby(level='Chain')["NAV"].sum(),
    'Protocol correctly identified': df[identified_filter].groupby(level='Chain').size().astype(int),
    'Stablecoin denominated': df[stablecoin_denominated].groupby(level='Chain').size().astype(int),
    f'Notable stablecoin NAV (min {nav_threshold} USD)': df[notable_nav].groupby(level='Chain').size().astype(int),
    f'ERC-7540': df[erc_7540].groupby(level='Chain').size().astype(int),
    f'ERC-7575': df[erc_7575].groupby(level='Chain').size().astype(int),
    f'Fee data supported': df[fee_detected].groupby(level='Chain').size().astype(int),
}).fillna(0)

# TODO: Having NA in calculations somewhere confuses Pandas and makes int columns floats even if the 
# NA is not present in the final results
print("Vault counts per feature per chain")
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Total vault TVL/NAV in stable vaults,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 100000 USD),ERC-7540,ERC-7575,Fee data supported
Chain,,,,,,,,
Arbitrum,1916,"122,413,434.00",342.00,659,55.00,18.00,26.00,219.00
Avalanche,280,"61,745,687.18",39.00,82,6.00,1.00,1.00,48.00
Base,1158,"291,055,583.51",581.00,422,48.00,26.00,12.00,94.00
Berachain,233,"321,753,866.85",29.00,17,11.00,0.00,0.00,2.00
Binance,316,"220,607,730.24",18.00,84,9.00,0.00,0.00,25.00
Ethereum,2123,"9,456,167,378.26",527.00,706,178.00,16.00,14.00,177.00
Hyperliquid,12,"214,705.42",0.00,6,1.00,1.00,0.00,1.00
Mantle,37,363.15,0.00,11,0.00,0.00,0.00,5.00
Mode,56,"449,480.98",2.00,26,2.00,0.00,0.00,0.00


## Vaults per protocol summary

- Break down by identified protocol
- Because of the brokeness of EVM, Solidity and smart contract development practices, protocol-rules are hand-maintained heurestics and there can false positives and negatives

In [5]:
df = source_df.copy()
df = df.reset_index()
# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True 
erc_7575 = df["ERC-7575"] == True 
fee_detected = df["Fee detected"] == True 

stablecoin_only_df = df[df["Stablecoin denominated"] == True]
stablecoin_only_df = stablecoin_only_df[stablecoin_only_df["NAV"] < broken_max_nav_threshold]
# Create the summary DataFrame
summary_df = pd.DataFrame({
    'Total vaults detected': df.groupby('Protocol').size(),
    'Total vault TVL/NAV in stable vaults': stablecoin_only_df.groupby('Protocol')["NAV"].sum(),
    'Protocol correctly identified': df[identified_filter].groupby('Protocol').size().astype(int),
    'Stablecoin denominated': df[stablecoin_denominated].groupby('Protocol').size().astype(int),
    f'Notable stablecoin NAV (min {nav_threshold} USD)': df[notable_nav].groupby('Protocol').size().astype(int),
    f'ERC-7540': df[erc_7540].groupby('Protocol').size().astype(int),
    f'ERC-7575': df[erc_7575].groupby('Protocol').size().astype(int),
    f'Fee data supported': df[fee_detected].groupby('Protocol').size().astype(int),
}).fillna(0)

# TODO: Having NA in calculations somewhere confuses Pandas and makes int columns floats even if the 
# NA is not present in the final results
print("Vault counts per feature per chain")
summary_df = summary_df.sort_values("Total vault TVL/NAV in stable vaults", ascending=False)
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Total vault TVL/NAV in stable vaults,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 100000 USD),ERC-7540,ERC-7575,Fee data supported
Protocol,,,,,,,,
<unknown ERC-4626>,4963,"8,322,541,930.95",0.00,"1,741.00",179.00,0.00,12.00,0.00
Morpho,183,"1,397,487,238.92",183.00,93.00,56.00,0.00,0.00,183.00
Fluid,57,"491,407,492.49",57.00,25.00,7.00,0.00,0.00,0.00
Euler Vault Kit,267,"164,962,943.06",267.00,68.00,20.00,0.00,0.00,0.00
Yearn v3,125,"68,535,496.08",125.00,63.00,18.00,0.00,0.00,0.00
Superform,106,"30,916,467.52",106.00,40.00,8.00,13.00,0.00,0.00
Kiln Metavault,69,"18,662,144.42",69.00,61.00,8.00,0.00,0.00,0.00
Yearn compounder,71,"16,684,963.12",71.00,12.00,1.00,0.00,0.00,0.00
Term Finance,16,"11,328,468.36",16.00,15.00,5.00,0.00,0.00,0.00


## Vault deployment history

- Show how much history we have for each chain


In [6]:
# Assuming your DataFrame is named 'df'
df = source_df
seen_df = df.groupby(level='Chain')['First seen'].agg(['min', 'max']).reset_index()

# Rename columns for clarity
seen_df.columns = ['Chain', 'First vault deployed', 'Last vault deployed']

seen_df = seen_df.set_index("Chain")

display(seen_df)

,First vault deployed,Last vault deployed
Chain,,
Arbitrum,2022-03-28 09:03:11,2025-04-03 20:38:18
Avalanche,2022-04-11 00:05:24,2025-04-10 00:26:55
Base,2023-08-04 15:36:07,2025-04-09 14:47:09
Berachain,2025-01-26 00:06:03,2025-04-09 07:50:02
Binance,2022-05-27 17:25:18,2025-04-08 11:29:07
Ethereum,2019-06-11 06:17:19,2025-04-09 17:13:47
Hyperliquid,2025-02-19 16:29:00,2025-04-07 15:02:00
Mantle,2023-08-22 13:25:16,2024-11-29 08:55:34
Mode,2024-03-06 23:58:43,2025-01-03 19:23:51


## Largest USD vaults

- Show the stablecoin-denominated vaults across different chains that have largest USD treasury 

In [7]:
largest_threshold = 20
largest_df = df.reset_index()
# Filter out crap
largest_df = largest_df[largest_df["Total events"] > 100] 
largest_df = largest_df[largest_df["Stablecoin denominated"] == True] 
largest_df = largest_df.sort_values(["NAV"], ascending=False)

largest_df = largest_df[["NAV", "Chain", "Address", "Name", "Denomination", "Total events", "Mgmt fee", "Perf fee"]]
largest_df = largest_df.set_index("Name")


display(largest_df.head(largest_threshold))


,NAV,Chain,Address,Denomination,Total events,Mgmt fee,Perf fee
Name,,,,,,,
Savings USDS,"2,979,613,394.52",Ethereum,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,USDS,28155,<unknown>,<unknown>
Staked USDe,"2,162,640,474.16",Ethereum,0x9d39a5de30e57443bff2a8307a4256c8797a3497,USDe,69230,<unknown>,<unknown>
Ethereal Pre-deposit Vault,"892,758,591.78",Ethereum,0x90d2af7d622ca3141efa4d8f1f24d86e5974cc8f,USDe,49121,<unknown>,<unknown>
Savings Dai,"518,495,409.71",Ethereum,0x83f20f44975d03b1b09e64809b757c47f942beea,DAI,72590,<unknown>,<unknown>
Bridged USDC (Stargate)Vault,"307,578,099.32",Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,135337,<unknown>,<unknown>
Usual Boosted USDC,"246,334,839.20",Ethereum,0xd63070114470f685b75b74d60eec7c1113d33a3d,USDC,17461,0.00,0.00
Fluid USD Coin,"210,501,860.29",Ethereum,0x9fb7b4477576fe5b32be4c1843afb1e55f251b33,USDC,17564,<unknown>,<unknown>
Fluid Tether USD,"200,522,647.05",Ethereum,0x5c20b550819128074fd538edf79791733ccedd18,USDT,10433,<unknown>,<unknown>
Staked USDX,"199,665,946.99",Binance,0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92,USDX,5449,<unknown>,<unknown>


## Largest USD vault per chain

- Get the largest vault of each chain

In [8]:
# Get the index of max NAV for each chain
largest_df = largest_df.reset_index().set_index(["Chain", "Name"])
max_nav_idx = largest_df.groupby('Chain')['NAV'].idxmax()
# Use these indices to get the full rows
result = largest_df.loc[max_nav_idx]

display(result)

,,NAV,Address,Denomination,Total events,Mgmt fee,Perf fee
Chain,Name,,,,,,
Arbitrum,Fluid USD Coin,"28,464,410.55",0x1a996cb54bb95462040408c06122d45d6cdb6096,USDC,33675,<unknown>,<unknown>
Avalanche,EVK Vault eUSDC-2,"21,664,340.50",0x39de0f00189306062d79edec6dca5bb6bfd108f9,USDC,349,<unknown>,<unknown>
Base,Seamless USDC Vault,"50,257,791.25",0x616a4e1db48e22028f6bbf20444cd3b8e3273738,USDC,79414,0.00,0.00
Berachain,Bridged USDC (Stargate)Vault,"307,578,099.32",0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,135337,<unknown>,<unknown>
Binance,Staked USDX,"199,665,946.99",0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92,USDX,5449,<unknown>,<unknown>
Ethereum,Savings USDS,"2,979,613,394.52",0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,USDS,28155,<unknown>,<unknown>
Hyperliquid,hyUSDXL (Looped HYPE) - 1,"214,704.91",0xaeedd5b6d42e0f077ccf3e7a78ff70b8cb217329,USDXL,111,<unknown>,<unknown>
Mode,USDC Ironclad Vault,"337,647.27",0x882fd369341fc435ad5e54e91d1ebc23b1fc6d4c,USDC,183,<unknown>,<unknown>
Polygon,Compound USDC,"23,436,257.49",0x781fb7f6d845e3be129289833b04d43aa8558c42,USDC,4251,0.00,0.00


## Most active vaults across all chains

- Determine vault activity by number of deposit and redeem events
- Based on all-time event count, not recent event count 
- Events may be driven by bots, so this may not reflect the popularity of a vault amount users


In [9]:
largest_threshold = 20
largest_df = df.reset_index().sort_values(["Total events"], ascending=False)

largest_df = largest_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

largest_df = largest_df.set_index("Name")

display(largest_df.head(largest_threshold))

,Total events,Chain,Address,Denomination,NAV,Age,Deposit count,Redeem count
Name,,,,,,,,
USDC yVault-A,490172,Polygon,0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0,USDC,"570,858.69",531 days 20:26:26.046174,465206,24966
USDT yVault-A,458210,Polygon,0xbb287e6017d3deb0e2e65061e8684eab21060123,USDT,"462,288.76",489 days 20:09:54.046174,407692,50518
pufETH,167878,Ethereum,0xd9a442856c234a39a81a089c06451ebaa4306a72,WETH,"69,830.82",434 days 17:27:39.046174,151577,16301
Moonwell Flagship ETH,140868,Base,0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1,WETH,"13,396.42",302 days 00:14:59.046174,85541,55327
Bridged USDC (Stargate)Vault,135337,Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,"307,578,099.32",74 days 13:06:11.046174,80977,54360
FARM_WETH,134078,Base,0x0b0193fad49de45f5e2b0a9f5d6bc3bb7d281688,WETH,"1,095.50",581 days 12:45:11.046174,76076,58002
Staked USDA,121775,Arbitrum,0x0022228a2cc5e7ef0274a7baa600d44da5ab5776,USDA,"967,132.98",455 days 04:39:49.046174,119754,2021
Beraborrow iBGT,100130,Berachain,0xe59ab0c3788217e48399dae3cd11929789e4d3b2,iBGT,"64,709.77",34 days 01:06:55.046174,53814,46316
Moonwell Flagship USDC,92471,Base,0xc1256ae5ff1cf2719d4937adb3bbccab2e00a2ca,USDC,"28,981,352.36",302 days 00:14:59.046174,53401,39070


## Most historically active vault per chain

- Vaults with most deposit and redeem events

In [10]:
most_active_df = df.reset_index()

most_active_df = most_active_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

# Force thousand separator
most_active_df["Total events"] = most_active_df["Total events"].astype("float64")

max_nav_idx = most_active_df.groupby('Chain')['Total events'].idxmax()
# Use these indices to get the full rows
result = most_active_df.loc[max_nav_idx]

result = result.set_index(["Chain", "Name"])

display(result)

,,Total events,Address,Denomination,NAV,Age,Deposit count,Redeem count
Chain,Name,,,,,,,
Arbitrum,Staked USDA,"121,775.00",0x0022228a2cc5e7ef0274a7baa600d44da5ab5776,USDA,"967,132.98",455 days 04:39:49.046174,119754,2021
Avalanche,GoGoPool Liquid Staking Token,"67,268.00",0xa25eaf2906fa1a3a13edac9b9657108af7b703e3,WAVAX,"860,520.18",735 days 16:01:29.046174,41541,25727
Base,Moonwell Flagship ETH,"140,868.00",0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1,WETH,"13,396.42",302 days 00:14:59.046174,85541,55327
Berachain,Bridged USDC (Stargate)Vault,"135,337.00",0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,"307,578,099.32",74 days 13:06:11.046174,80977,54360
Binance,kUSDT,"42,810.00",0x1c3f35f7883fc4ea8c4bca1507144dc6087ad0fb,VUSD,"2,593,079.58",668 days 06:03:31.046174,25334,17476
Ethereum,pufETH,"167,878.00",0xd9a442856c234a39a81a089c06451ebaa4306a72,WETH,"69,830.82",434 days 17:27:39.046174,151577,16301
Hyperliquid,wHYPE,664.00,0x2831775cb5e64b1d892853893858a261e898fbeb,WHYPE,"237,219.39",28 days 15:01:14.046174,513,151
Mantle,Karak - mETH,"13,028.00",0x8529019503c5bd707d8eb98c5c87bf5237f89135,mETH,488.97,351 days 19:00:08.046174,8038,4990
Mode,Renzo aggregator,"21,421.00",0xd60dd6981ec336fda40820f8ca5e99cd17dd25a0,WETH,213.21,399 days 13:13:31.046174,12188,9233


## Oldest vaults

- Show oldest vaults

In [11]:
threshold = 1_000

oldest_df = df.reset_index()

oldest_df = oldest_df[["Chain", "Address", "Name", "Age", "Denomination", "NAV", "Total events"]]

# Force thousand separator
oldest_df["Total events"] = oldest_df["Total events"].astype("float64")

# Force event threshold to filter out some crap
oldest_df = oldest_df[oldest_df["Total events"] >= threshold]

max_nav_idx = oldest_df.groupby('Chain')['Age'].idxmax()
# Use these indices to get the full rows
result = oldest_df.loc[max_nav_idx]

result = result.set_index("Chain")

display(result)

,Address,Name,Age,Denomination,NAV,Total events
Chain,,,,,,
Arbitrum,0xf46ce0c13577232d5f29d9bd78a9cab278755346,Jones ETH,1048 days 21:57:08.046174,WETH,18.00,"2,335.00"
Avalanche,0x9dd17f32fc8355ae37425f475a10cc7bec8ca36a,,1092 days 20:14:52.046174,,0.00,"1,425.00"
Base,0xc7548d8d7560f6679e369d0556c44fe1eddea3e9,FARM_WETH,587 days 21:18:21.046174,WETH,0.97,"1,117.00"
Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,Bridged USDC (Stargate)Vault,74 days 13:06:11.046174,USDC.e,"307,578,099.32","135,337.00"
Binance,0x0f8754b36a767c5579178bd8a04d2fcd9d530b70,ygNRCH,1017 days 16:19:35.046174,NRCH,"1,188,908.73","1,195.00"
Ethereum,0x815c23eca83261b6ec689b60cc4a58b54bc24d8d,vTHOR,1081 days 01:46:37.046174,THOR,"79,580,695.56","19,579.00"
Mantle,0x8529019503c5bd707d8eb98c5c87bf5237f89135,Karak - mETH,351 days 19:00:08.046174,mETH,488.97,"13,028.00"
Mode,0xd60dd6981ec336fda40820f8ca5e99cd17dd25a0,Renzo aggregator,399 days 13:13:31.046174,WETH,213.21,"21,421.00"
Polygon,0x73958d46b7aa2bc94926d8a215fa560a5cdca3ea,Wrapped Aave Polygon GHST,1072 days 09:25:25.046174,aPolGHST,"1,144,894.44","14,514.00"


## All vaults

- List of all stablecoin vaults

In [12]:
all_df = df.reset_index()

min_nav = 10_000

stablecoin_denominated = all_df["Stablecoin denominated"] == True
all_df = all_df[stablecoin_denominated]
all_df = all_df[all_df["NAV"] < broken_max_nav_threshold]
all_df = all_df[all_df["NAV"] >= min_nav]
all_df = all_df[["NAV", "Chain", "Protocol", "Address", "Name", "Age", "Denomination", "Total events"]]
all_df = all_df.set_index("Name").sort_values("NAV", ascending=False)

print(f"Displaying {len(all_df)} stablecoin vaults w/min NAV of {min_nav} USD")
pd.options.display.max_rows = None
display(all_df)


Displaying 496 stablecoin vaults w/min NAV of 10000 USD


,NAV,Chain,Protocol,Address,Age,Denomination,Total events
Name,,,,,,,
Savings USDS,"2,979,613,394.52",Ethereum,<unknown ERC-4626>,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,204 days 23:35:27.046174,USDS,28155
Staked USDe,"2,162,640,474.16",Ethereum,<unknown ERC-4626>,0x9d39a5de30e57443bff2a8307a4256c8797a3497,506 days 05:06:03.046174,USDe,69230
Ethereal Pre-deposit Vault,"892,758,591.78",Ethereum,<unknown ERC-4626>,0x90d2af7d622ca3141efa4d8f1f24d86e5974cc8f,55 days 21:58:03.046174,USDe,49121
Savings Dai,"518,495,409.71",Ethereum,<unknown ERC-4626>,0x83f20f44975d03b1b09e64809b757c47f942beea,743 days 02:47:27.046174,DAI,72590
Spark DAI Vault,"325,124,600.68",Ethereum,Morpho,0x73e65dbd630f90604062f6e02fab9138e713edd9,376 days 22:52:27.046174,DAI,96
Bridged USDC (Stargate)Vault,"307,578,099.32",Berachain,<unknown ERC-4626>,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,74 days 13:06:11.046174,USDC.e,135337
Usual Boosted USDC,"246,334,839.20",Ethereum,Morpho,0xd63070114470f685b75b74d60eec7c1113d33a3d,258 days 18:18:03.046174,USDC,17461
Fluid USD Coin,"210,501,860.29",Ethereum,Fluid,0x9fb7b4477576fe5b32be4c1843afb1e55f251b33,416 days 11:56:51.046174,USDC,17564
Fluid Tether USD,"200,522,647.05",Ethereum,Fluid,0x5c20b550819128074fd538edf79791733ccedd18,416 days 11:56:51.046174,USDT,10433
